In [2]:
import pandas as pd
import yfinance as yf
import numpy as np
import requests
import time
from yahooquery import Ticker


# ==========================================
# 1. STEP 1: FETCH CANADIAN UNIVERSE
# ==========================================
# ==========================================
# 1. STEP 1: FETCH CANADIAN UNIVERSE (ROBUST)
# ==========================================
def get_canadian_universe():
    print("--- STEP 1: Fetching TSX & TSX-V Stock List ---")
    tickers = []
    
    # 1. Try Fetching from TMX with "Browser Headers"
    url = "https://www.tsx.com/files/trading/moc-eligible-stocks.txt"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            lines = response.text.split('\n')
            for line in lines:
                parts = line.split()
                if len(parts) < 2: continue
                
                exchange = parts[0]
                symbol = parts[1]
                
                if exchange == 'TSX':
                    tickers.append(f"{symbol.replace('.', '-')}.TO")
                elif exchange == 'TSXV':
                    tickers.append(f"{symbol.replace('.', '-')}.V")
                    
        print(f"   -> Scraper found {len(tickers)} candidates.")
    except Exception as e:
        print(f"   -> Web scrape failed ({e}). Switching to backup list...")

    # 2. SAFETY NET: If scraper failed or returned 0, use this Manual List
    if len(tickers) == 0:
        print("   -> Using Backup List of Top Canadian Financials/REITs...")
        # A mix of Banks, Insurance, Asset Managers, and liquid REITs
        tickers = [
            # Big 6 Banks
            'RY.TO', 'TD.TO', 'BMO.TO', 'BNS.TO', 'CM.TO', 'NA.TO',
            # Other Financials
            'MFC.TO', 'SLF.TO', 'GWO.TO', 'POW.TO', 'FFH.TO', 'IAG.TO', 
            'CWB.TO', 'LB.TO', 'EQB.TO', 'X.TO', 'CF.TO', 'ONEX.TO', 
            'CIX.TO', 'EFN.TO', 'MIC.TO', 'FN.TO',
            # REITs (often categorized as Real Estate/Financial)
            'REI-UN.TO', 'CAR-UN.TO', 'DIR-UN.TO', 'GRT-UN.TO', 'AP-UN.TO', 
            'SRU-UN.TO', 'HR-UN.TO', 'KMP-UN.TO', 'CRT-UN.TO', 'WIR-UN.TO'
        ]

    return list(set(tickers))

# ==========================================
# 2. STEP 2: FINANCIALS ONLY FILTER (CANADA)
# ==========================================
def filter_financials_universe(ticker_list):
    print(f"\n--- STEP 2: Filtering for Canadian Financials (YahooQuery) ---")
    
    # --- NEW FILTERS ---
    MIN_PRICE = 3.0       # Slightly lower for Canada
    MIN_CAP = 100_000_000 # > $100M
    MIN_VOL = 100_000     # > 100k Volume
    
    valid_candidates = []
    chunk_size = 500
    
    for i in range(0, len(ticker_list), chunk_size):
        chunk = ticker_list[i:i+chunk_size]
        print(f"   Filtering batch {i} - {min(i+chunk_size, len(ticker_list))}...", end='\r')
        
        try:
            yq = Ticker(chunk, asynchronous=False)
            # added 'summaryDetail' to get volume/marketCap
            data = yq.get_modules("summaryProfile summaryDetail price financialData")
            
            for symbol in chunk:
                if symbol not in data or isinstance(data[symbol], str): continue
                
                # 1. Sector Check
                profile = data[symbol].get('summaryProfile', {})
                sector = profile.get('sector', 'Unknown')
                if 'Financial' not in sector and 'Real Estate' not in sector: continue

                # 2. Basic Data (Price, Cap, Vol)
                price_mod = data[symbol].get('price', {})
                curr_price = price_mod.get('regularMarketPrice', 0) or 0
                mkt_cap = price_mod.get('marketCap', 0) or 0
                
                summ_mod = data[symbol].get('summaryDetail', {})
                avg_vol = summ_mod.get('averageVolume', 0) or 0
                
                fin_mod = data[symbol].get('financialData', {})
                rec_key = fin_mod.get('recommendationKey', 'none')
                if rec_key and isinstance(rec_key, str): rec_key = rec_key.lower().strip()
                
                # 3. Apply Filters
                if curr_price < MIN_PRICE: continue
                if mkt_cap < MIN_CAP: continue
                if avg_vol < MIN_VOL: continue  # <-- Added Volume Filter
                
                # (Optional) Loose Rating Filter: Allow 'hold' for Canada as analysts are stricter
                if rec_key not in ['buy', 'strong_buy', 'strong buy', 'hold']: continue

                valid_candidates.append({
                    'Ticker': symbol, 
                    'Price': curr_price,
                    'Sector': sector,
                    'Rating': rec_key
                })
        except:
            continue
            
    df = pd.DataFrame(valid_candidates)
    print(f"\n   -> Filter complete. Survivors: {len(df)}")
    return df

# ==========================================
# 3. DATA ENRICHMENT
# ==========================================
def fetch_financial_data(df):
    if df is None or df.empty:
        print("No financial stocks to analyze.")
        return None

    print(f"\n--- STEP 3: FETCHING METRICS FOR {len(df)} STOCKS ---")
    bank_data = []
    
    for index, row in df.iterrows():
        ticker = row['Ticker']
        if index % 5 == 0: print(f"   Fetching {index}/{len(df)}...", end='\r')
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            pe = info.get('trailingPE', np.nan)
            pb = info.get('priceToBook', np.nan)
            roe = info.get('returnOnEquity', np.nan)
            div_yield = info.get('dividendYield', 0)
            if div_yield is None: div_yield = 0
            
            recom = info.get('recommendationMean', None)
            
            bank_data.append({
                'Ticker': ticker,
                'Price': row['Price'],
                'P/E': pe,
                'P/B': pb,
                'ROE': roe,
                'Yield%': round(div_yield, 2),
                'Recom': float(recom) if recom is not None else 3.0,
                'Sector': row.get('Sector', 'Financial')
            })
        except: continue
            
    master_df = pd.DataFrame(bank_data)
    
    # Force numeric types
    cols = ['P/E', 'P/B', 'ROE', 'Yield%', 'Recom']
    for col in cols: master_df[col] = pd.to_numeric(master_df[col], errors='coerce')
    
    return master_df

# ==========================================
# 4. VIEW: VALUE PICKS
# ==========================================
def get_value_picks(df):
    # Filter: P/E < 15, P/B < 1.2, ROE > 8%
    mask = (df['P/E'] < 15) & (df['P/B'] < 1.2) & (df['ROE'] > 0.08)
    value_df = df[mask].copy()
    value_df = value_df.sort_values(by='P/B', ascending=True)
    
    cols = ['Ticker', 'Price', 'P/B', 'P/E', 'ROE', 'Yield%', 'Recom']
    return value_df[cols]

# ==========================================
# 5. VIEW: INCOME PICKS
# ==========================================
def get_income_picks(df):
    # Filter: Yield > 2.5%, P/E < 20, Rating <= 2.5 (Buy/Hold)
    mask = (df['Yield%'] >= 2.5) & (df['P/E'] < 20) & (df['Recom'] <= 2.5)
    income_df = df[mask].copy()
    income_df = income_df.sort_values(by='Yield%', ascending=False)
    
    cols = ['Ticker', 'Price', 'Yield%', 'P/E', 'Recom', 'Sector']
    return income_df[cols]

# ==========================================
# EXECUTION BLOCK
# ==========================================
raw_tickers = get_canadian_universe()
financial_df = filter_financials_universe(raw_tickers)

if not financial_df.empty:
    master_df = fetch_financial_data(financial_df)
    
    if master_df is not None and not master_df.empty:
        # Create the variables for Data Wrangler
        df_value_ca = get_value_picks(master_df)
        df_income_ca = get_income_picks(master_df)
        
        print("\n\n✅ Done! Two Canadian DataFrames are ready:")
        print("   1. df_value_ca  (Undervalued)")
        print("   2. df_income_ca (High Dividend)")
        
        print("\n--- Value Preview (Canada) ---")
        print(df_value_ca.head())

--- STEP 1: Fetching TSX & TSX-V Stock List ---
   -> Scraper found 0 candidates.
   -> Using Backup List of Top Canadian Financials/REITs...

--- STEP 2: Filtering for Canadian Financials (YahooQuery) ---
   Filtering batch 0 - 32...
   -> Filter complete. Survivors: 15

--- STEP 3: FETCHING METRICS FOR 15 STOCKS ---
   Fetching 10/15...

✅ Done! Two Canadian DataFrames are ready:
   1. df_value_ca  (Undervalued)
   2. df_income_ca (High Dividend)

--- Value Preview (Canada) ---
      Ticker  Price       P/B       P/E      ROE  Yield%    Recom
8  KMP-UN.TO  16.27  0.610827  3.623608  0.18095    4.43  1.72727
